In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import plotly.express as px
import seaborn as sns
!pip install category-encoders
from category_encoders import OrdinalEncoder
from sklearn.linear_model import LogisticRegression


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
data = pd.read_csv('./data/sf-crime/train.csv')

columns_deleted = ['Descript','Resolution','Address', 'PdDistrict']
data = data.drop(columns_deleted, axis=1)
print(data.head())

# Les valeurs connues pour la suppression
# category_a_supprimer = 'WARRANTS'
x_a_supprimer = -120.5
y_a_supprimer = 90

# Supprimer les lignes où les valeurs correspondent aux valeurs connues
# data = data[~((data['Category'] == category_a_supprimer) & (data['X'] == x_a_supprimer) & (data['Y'] == y_a_supprimer))]
data = data[~((data['X'] == x_a_supprimer) & (data['Y'] == y_a_supprimer))]

categories_uniques = data['Category'].unique()
print(categories_uniques)
# Définir les catégories
categories = {
    "Crimes against the person": [
        'ASSAULT', 'ROBBERY', 'KIDNAPPING', 'SEX OFFENSES FORCIBLE',
        'SEX OFFENSES NON FORCIBLE', 'HOMICIDE'
    ],
    "Crimes against property": [
        'LARCENY/THEFT', 'VEHICLE THEFT', 'BURGLARY', 'VANDALISM',
        'STOLEN PROPERTY', 'RECOVERED VEHICLE', 'ARSON', 'FRAUD',
        'EMBEZZLEMENT', 'EXTORTION', 'BRIBERY', 'FORGERY/COUNTERFEITING',
        'BAD CHECKS'
    ],
    "Crimes related to alcohol and drugs": [
        'DRUNKENNESS', 'DRUG/NARCOTIC', 'LIQUOR LAWS','DRIVING UNDER THE INFLUENCE'
    ],
    "Crimes related to public order": [
        'DISORDERLY CONDUCT', 'TRESPASS', 'LOITERING', 'SUSPICIOUS OCC','FAMILY OFFENSES'
    ],
    "Miscellaneous": [
        'WARRANTS', 'OTHER OFFENSES', 'NON-CRIMINAL', 'SECONDARY CODES',
        'MISSING PERSON', 'RUNAWAY', 'GAMBLING', 'PORNOGRAPHY/OBSCENE MAT',
        'TREA'
    ]
}
# Créer un dictionnaire inversé pour obtenir les catégories pour chaque thème
theme_categories = {}
for category, themes_list in categories.items():
    for theme in themes_list:
        theme_categories[theme] = category

print(theme_categories)

# Utiliser le dictionnaire 'theme_categories' pour mapper chaque donnée à sa catégorie
data['Category'] = data['Category'].map(theme_categories)

# Randomiser les données
data = data.sample(frac=1).reset_index(drop=True)

# # Compter le nombre de données pour chaque catégorie
counts_per_category = data['Category'].value_counts()
#
# # Trouver la taille de la catégorie la plus petite
min_count = counts_per_category.min()
#
# # Sélectionner un nombre équilibré de données pour chaque catégorie (basé sur la taille de la catégorie la plus petite)
balanced_data = pd.DataFrame()
#
for category, count in counts_per_category.items():
    category_data = data[data['Category'] == category].sample(min(count, min_count))
    balanced_data = pd.concat([balanced_data, category_data])
#
# # Enregistrer ce nouvel ensemble de données équilibré dans un fichier CSV
balanced_data.to_csv('./data/sf-crime//balanced_dataset.csv', index=False)
#
data = pd.read_csv('./data/sf-crime/balanced_dataset.csv')

# Mapper les catégories à partir des valeurs existantes de la colonne 'Category'
data['Category'] = data['Category'].map(theme_categories).fillna(data['Category'])
# Convertir la colonne 'Dates' en format datetime
data['Dates'] = pd.to_datetime(data['Dates'])
# Créer de nouvelles colonnes pour la date et l'heure
data['Date'] = data['Dates'].dt.date
data['Time'] = data['Dates'].dt.time
# Supprimer la colonne 'Dates' maintenant qu'elle n'est plus nécessaire
data.drop(columns=['Dates'], inplace=True)
# Convertir la colonne 'Date' en composantes de date séparées
data['Year'] = pd.to_datetime(data['Date']).dt.year
data['Month'] = pd.to_datetime(data['Date']).dt.month
data['Day'] = pd.to_datetime(data['Date']).dt.day
# Convertir la colonne 'Time' en composantes d'heure séparées
data['Hour'] = pd.to_datetime(data['Time'], format='%H:%M:%S').dt.hour
data['Minute'] = pd.to_datetime(data['Time'], format='%H:%M:%S').dt.minute
data['Second'] = pd.to_datetime(data['Time'], format='%H:%M:%S').dt.second
# Supprimer les colonnes d'origine 'Date' et 'Time'
data.drop(['Date', 'Time'], axis=1, inplace=True)
# Colonnes à encoder de façon ordinaire
ordinal_cols = ['Category', 'DayOfWeek']

print(data.isnull().sum())

# Créer un encodeur ordinal et appliquer la transformation
encoder = OrdinalEncoder(cols=ordinal_cols).fit(data)
data = encoder.transform(data)
# Enregistrer le nouveau fichier CSV sans modifier l'original
data.to_csv('./data/sf-crime/nouveau_fichier.csv', index=False)
# Filtrer les données pour inclure uniquement la catégorie 'Harassment'
larceny_theft_data = data[data['Category'] == 'Harassment']
# Créer un nuage de points avec la catégorie 'PROSTITUTION'
# fig = px.scatter(larceny_theft_data, x="X", y="Y", color="Category")
# fig.show()
# Vérifier s'il y a des valeurs manquantes (NaN) dans chaque colonne
missing_values = data.isnull().any()
data.isnull().sum()
# Afficher les colonnes contenant des valeurs manquantes (True) et le nombre de valeurs manquantes dans chaque colonne
print(missing_values)
print(data.isnull().sum())
print(data)
print(data.shape)

#

                 Dates        Category  DayOfWeek           X          Y
0  2015-05-13 23:53:00        WARRANTS  Wednesday -122.425892  37.774599
1  2015-05-13 23:53:00  OTHER OFFENSES  Wednesday -122.425892  37.774599
2  2015-05-13 23:33:00  OTHER OFFENSES  Wednesday -122.424363  37.800414
3  2015-05-13 23:30:00   LARCENY/THEFT  Wednesday -122.426995  37.800873
4  2015-05-13 23:30:00   LARCENY/THEFT  Wednesday -122.438738  37.771541
['WARRANTS' 'OTHER OFFENSES' 'LARCENY/THEFT' 'VEHICLE THEFT' 'VANDALISM'
 'NON-CRIMINAL' 'ROBBERY' 'ASSAULT' 'WEAPON LAWS' 'BURGLARY'
 'SUSPICIOUS OCC' 'DRUNKENNESS' 'FORGERY/COUNTERFEITING' 'DRUG/NARCOTIC'
 'STOLEN PROPERTY' 'SECONDARY CODES' 'TRESPASS' 'MISSING PERSON' 'FRAUD'
 'KIDNAPPING' 'RUNAWAY' 'DRIVING UNDER THE INFLUENCE'
 'SEX OFFENSES FORCIBLE' 'PROSTITUTION' 'DISORDERLY CONDUCT' 'ARSON'
 'FAMILY OFFENSES' 'LIQUOR LAWS' 'BRIBERY' 'EMBEZZLEMENT' 'SUICIDE'
 'LOITERING' 'SEX OFFENSES NON FORCIBLE' 'EXTORTION' 'GAMBLING'
 'BAD CHECKS' 'TREA' 'RECOV

In [13]:
####### Model 1 #######
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Sélection des colonnes souhaitées pour les entrées du modèle
selected_columns = ['DayOfWeek', 'X', 'Y', 'Hour']
X = data[selected_columns]
y = data['Category']
#
# # Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#
# # Create a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=15, random_state=42,max_depth=20)

# Entraînement du modèle
clf.fit(X_train, y_train)

# Validation croisée (cross-validation)
scores = cross_val_score(clf, X, y, cv=5)  # Utilisation de 5 folds pour la validation croisée

# Affichage des scores de validation croisée
print("Scores de validation croisée :", scores)
print("Moyenne des scores :", scores.mean())

# Prédictions sur l'ensemble de test
predictions = clf.predict(X_test)

# Évaluation du modèle sur l'ensemble de test
print("Accuracy :", clf.score(X_test, y_test))
print("Confusion Matrix :\n", confusion_matrix(y_test, predictions))
print("Classification Report :\n", classification_report(y_test, predictions))

# Nouvelles données à prédire
new_data = [
    [4, 37.7749, -122.4194, 8],
    [2, 37.7996, -122.4000, 4]
]

# Obtention des probabilités des prédictions pour les nouvelles données
predicted_probabilities = clf.predict_proba(new_data)[0]

categories_uniques = data['Category'].unique()

# Affichage des probabilités de chaque catégorie
for i, category in enumerate(categories_uniques):
    print(f"Prédiction de la catégorie {category} la plus probable : {predicted_probabilities[i] * 100:.2f}%")

Scores de validation croisée : [0.35739653 0.35563199 0.36483438 0.36009917 0.36166268]
Moyenne des scores : 0.35992495141944564
Accuracy : 0.35844364697648795
Confusion Matrix :
 [[5155 2420 2282 1100 2399]
 [2598 3299 2364 2575 2434]
 [2623 2453 4024 1972 2439]
 [1181 1783 1557 7679 1296]
 [2848 2561 2541 1659 3915]]
Classification Report :
               precision    recall  f1-score   support

           1       0.36      0.39      0.37     13356
           2       0.26      0.25      0.26     13270
           3       0.32      0.30      0.31     13511
           4       0.51      0.57      0.54     13496
           5       0.31      0.29      0.30     13524

    accuracy                           0.36     67157
   macro avg       0.35      0.36      0.35     67157
weighted avg       0.35      0.36      0.36     67157

Prédiction de la catégorie 1 la plus probable : 20.00%
Prédiction de la catégorie 2 la plus probable : 20.00%
Prédiction de la catégorie 3 la plus probable : 26.67%


C:\Users\Abdel\IdeaProjects\goyave-back\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [15]:
####### Sauvegarde du Model 1 #######

import pickle

# Sauvegarde du modèle dans un fichier
with open('crime_prediction_model.pkl', 'wb') as file:
    pickle.dump(clf, file)